In [16]:
!pip install seleniumbase

     ---------------------------------------- 0.0/86.0 kB ? eta -:--:--
     --------- ---------------------------- 20.5/86.0 kB 330.3 kB/s eta 0:00:01
     -------------------------------------- 86.0/86.0 kB 806.4 kB/s eta 0:00:00
  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
   ---------------------------------------- 0.0/618.7 kB ? eta -:--:--
   ------- -------------------------------- 122.9/618.7 kB 3.6 MB/s eta 0:00:01
   ----------- ---------------------------- 184.3/618.7 kB 2.2 MB/s eta 0:00:01
   --------------- ------------------------ 245.8/618.7 kB 1.9 MB/s eta 0:00:01
   ------------------- -------------------- 307.2/618.7 kB 1.9 MB/s eta 0:00:01
   ----------------------- ---------------- 368.6/618.7 kB 1.9 MB/s eta 0:00:01
   --------------------------- ------------ 430.1/618.7 kB 1.8 MB/s eta 0:00:01
   ---------------------------------------  614.4/618.7 kB 1.9 MB/s eta 0:00:01
   ---------------------------------------- 618.7/618.7 kB 1.9 MB/s eta 0:

ERROR: To modify pip, please run the following command:
C:\Users\RoG\anaconda3\envs\licenta\python.exe -m pip install seleniumbase


In [19]:
!sbase install chromedriver

'sbase' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url = "https://www.litcharts.com/lit/heart-of-darkness/part-1"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}

In [8]:
response = requests.get(url, headers=headers)
print(response.status_code)
soup = BeautifulSoup(response.text, "html.parser")

summaries = soup.find_all("div", class_="summary-text readable highlightable-content non-paywall")
analysis = soup.find_all("div", class_="analysis-text highlightable-content non-paywall")


200


In [9]:
summary_texts = [summary.text.strip() for summary in summaries]
analysis_texts = [analyse.text.strip() for analyse in analysis]
HoD = pd.DataFrame({"Summary": summary_texts, "Analysis": analysis_texts})
HoD.to_csv("HoD.csv", index=False)


In [10]:
# print first row of the dataframe
print(HoD.head(1))

                                             Summary  \
0  The Narrator describes the scene from the deck...   

                                            Analysis  
0  The opening establishes a dark tone. Water is ...  


In [12]:
part_index = 0

while True:
    part_index += 1
    url = f"https://www.litcharts.com/lit/heart-of-darkness/part-{part_index}"
    response = requests.get(url, headers=headers)
    print(response.status_code)
    if response.status_code != 200:
        break
    soup = BeautifulSoup(response.text, "html.parser")
    summaries = soup.find_all("div", class_="summary-text readable highlightable-content non-paywall")
    analysis = soup.find_all("div", class_="analysis-text highlightable-content non-paywall")
    summary_texts = [summary.text.strip() for summary in summaries]
    analysis_texts = [analyse.text.strip() for analyse in analysis]
    HoD = pd.DataFrame({"Summary": summary_texts, "Analysis": analysis_texts})
    HoD.to_csv("HoD.csv", mode='a', header=False, index=False)

202


In [13]:
print(HoD.head(1))

                                             Summary  \
0  The Narrator describes the scene from the deck...   

                                            Analysis  
0  The opening establishes a dark tone. Water is ...  


In [3]:
import requests
from bs4 import BeautifulSoup

# URL of the page
url = "https://www.litcharts.com/lit/all-my-sons"  # Replace with the actual page URL
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Send a request to the page
response = requests.get(url, headers=headers)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    
    dropdown_div = soup.find('div', class_='component dropdown summary-sections')
    #print(dropdown_div)

    # Locate the dropdown menu
    dropdown_menu = dropdown_div.find('ul', class_='dropdown-menu')
    
    if dropdown_menu:
        # Extract all <a> tags inside the dropdown menu
        dropdown_items = dropdown_menu.find_all('a')
        
        # Get the text from each <a> tag
        acts = [item.get_text(strip=True) for item in dropdown_items]
        print(acts)  # Output: ['Act 1', 'Act 2', 'Act 3']
    else:
        print("Dropdown menu not found on the page.")
else:
    print(f"Failed to fetch page. Status code: {response.status_code}")


Failed to fetch page. Status code: 202


Scraper

In [8]:
import re
def get_summary_sections(url):
    response = requests.get(url, headers=headers)
    print(response.status_code)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        dropdown_div = soup.find('div', class_='component dropdown summary-sections')
        if dropdown_div:
            dropdown_menu = dropdown_div.find('ul', class_='dropdown-menu')
            if dropdown_menu:
                dropdown_items = dropdown_menu.find_all('a')
                sections = [
                            re.sub(r'[^\w\s-]', '', item.get_text(strip=True).lower()).replace(" ", "-")
                            for item in dropdown_items
                        ]       
                return sections
            else:
                return None
        else:
            return None
    else:
        return None

In [5]:
url = "https://www.litcharts.com/lit/heart-of-darkness"
sections = get_summary_sections(url)
print(sections)

202
None


In [4]:
import time
def get_summary_and_analysis(title, timeout=30, max_parts=0):
    url = f"https://www.litcharts.com/lit/{title}"
    sections = get_summary_sections(url)
    time.sleep(timeout)
    if sections:
        index = 0
        if max_parts == 0:
            max_parts = len(sections)
        for section in sections:
            if index == max_parts:
                break
            index += 1
            section_url = f"{url}/{section}"
            response = requests.get(section_url, headers=headers)
            time.sleep(timeout)
            print(response.status_code)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, "html.parser")
                summaries = soup.find_all("div", class_="summary-text readable highlightable-content non-paywall")
                analysis = soup.find_all("div", class_="analysis-text highlightable-content non-paywall")
                summary_texts = [summary.text.strip() for summary in summaries]
                analysis_texts = [analyse.text.strip() for analyse in analysis]
                testDF = pd.DataFrame({"Summary": summary_texts, "Analysis": analysis_texts, "Title": title * len(summary_texts)})
                testDF.to_csv("test.csv", mode='a', header=False, index=False)
            elif response.status_code == 202:
                time.sleep(timeout)
                response = requests.get(section_url, headers=headers)
                print(response.status_code)
                if response.status_code == 200:
                    soup = BeautifulSoup(response.text, "html.parser")
                    summaries = soup.find_all("div", class_="summary-text readable highlightable-content non-paywall")
                    analysis = soup.find_all("div", class_="analysis-text highlightable-content non-paywall")
                    summary_texts = [summary.text.strip() for summary in summaries]
                    analysis_texts = [analyse.text.strip() for analyse in analysis]
                    testDF = pd.DataFrame({"Summary": summary_texts, "Analysis": analysis_texts, "Title": title * len(summary_texts)})
                    testDF.to_csv("test.csv", mode='a', header=False, index=False)
            else:
                break

In [9]:
title = "the-idiot"
get_summary_and_analysis(title, timeout=60)

200
200
200
200
200
200
202


Selenium

In [28]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time

# Set up Selenium WebDriver
# service = Service('D:\Chromedriver\chromedriver-win64')
driver = webdriver.Chrome()

def get_summary_sections(url):
    driver.get(url)
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    dropdown_div = soup.find('div', class_='component dropdown summary-sections')
    if dropdown_div:
        dropdown_menu = dropdown_div.find('ul', class_='dropdown-menu')
        if dropdown_menu:
            dropdown_items = dropdown_menu.find_all('a')
            sections = [item.get('href') for item in dropdown_items if item.get('href')]    
            return sections
    return None

def get_summary_and_analysis(title, timeout=30, max_parts=0):
    url = f"https://www.litcharts.com/lit/{title}"
    sections = get_summary_sections(url)
    time.sleep(timeout)
    if sections:
        index = 0
        if max_parts == 0:
            max_parts = len(sections)
        for section in sections:
            if index == max_parts:
                break
            index += 1
            print(f"Getting data for section: {section}")
            section_url = f"https://www.litcharts.com{section}"
            driver.get(section_url)
            time.sleep(5) 
            soup = BeautifulSoup(driver.page_source, "html.parser")
            summaries = soup.find_all("div", class_="summary-text readable highlightable-content non-paywall")
            analysis = soup.find_all("div", class_="analysis-text highlightable-content non-paywall")
            summary_texts = [summary.text.strip() for summary in summaries]
            analysis_texts = [analyse.text.strip() for analyse in analysis]
            
            if summary_texts and analysis_texts:
                testDF = pd.DataFrame({
                    "Summary": summary_texts,
                    "Analysis": analysis_texts,
                    "Title": [title] * len(summary_texts)
                })
                testDF.to_csv("test_selenium.csv", mode='a', header=False, index=False)
            else:
                print(f"No summaries or analysis found for section: {section}")

try:
    get_summary_and_analysis("the-catcher-in-the-rye", timeout=10, max_parts=0)
finally:
    driver.quit()  


Getting data for section: /lit/the-catcher-in-the-rye/chapter-1
Getting data for section: /lit/the-catcher-in-the-rye/chapter-2
Getting data for section: /lit/the-catcher-in-the-rye/chapter-3
Getting data for section: /lit/the-catcher-in-the-rye/chapter-4
Getting data for section: /lit/the-catcher-in-the-rye/chapter-5
Getting data for section: /lit/the-catcher-in-the-rye/chapter-6
Getting data for section: /lit/the-catcher-in-the-rye/chapter-7
Getting data for section: /lit/the-catcher-in-the-rye/chapter-8
Getting data for section: /lit/the-catcher-in-the-rye/chapter-9
Getting data for section: /lit/the-catcher-in-the-rye/chapter-10
Getting data for section: /lit/the-catcher-in-the-rye/chapter-11
Getting data for section: /lit/the-catcher-in-the-rye/chapter-12
Getting data for section: /lit/the-catcher-in-the-rye/chapter-13
Getting data for section: /lit/the-catcher-in-the-rye/chapter-14
Getting data for section: /lit/the-catcher-in-the-rye/chapter-15
Getting data for section: /lit/the

No chapter scraper

In [22]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time

# Set up Selenium WebDriver
# service = Service('D:\Chromedriver\chromedriver-win64')
driver = webdriver.Chrome()

def get_summary_and_analysis_no_chapters(title, timeout=30):
    print(f"Getting data for section: {title}")
    section_url = f"https://www.litcharts.com/lit/{title}/summary-and-analysis"
    driver.get(section_url)
    time.sleep(5) 
    soup = BeautifulSoup(driver.page_source, "html.parser")
    summaries = soup.find_all("div", class_="summary-text readable highlightable-content non-paywall")
    analysis = soup.find_all("div", class_="analysis-text highlightable-content non-paywall")
    summary_texts = [summary.text.strip() for summary in summaries]
    analysis_texts = [analyse.text.strip() for analyse in analysis]
    
    if summary_texts and analysis_texts:
        testDF = pd.DataFrame({
            "Summary": summary_texts,
            "Analysis": analysis_texts,
            "Title": [title] * len(summary_texts)
        })
        testDF.to_csv("test_selenium.csv", mode='a', header=False, index=False)
    else:
        print(f"No summaries or analysis found for section: {title}")

try:
    get_summary_and_analysis_no_chapters("the-killers", timeout=10)
finally:
    driver.quit()  


Getting data for section: the-killers


In [31]:
import pandas as pd

# Load the CSV file without headers
df = pd.read_csv("test_selenium.csv", header=None)

# Assign column names (replace with your actual column names)
df.columns = ["Summary", "Analysis", "Title"]

# Save the updated CSV with headers
df.to_csv("test_selenium.csv", index=False)
